# $\color{black}{\text{Kapittel 1 - Grunnleggende regning}}$

In [ ]:
#1 Grunnleggende regning: 1.1 Hoderegning
# Addisjon, subtraksjon, multiplikasjon og divisjon

# Addisjon
addition_1 = 70 + 50
addition_2 = 700 + 500

# Subtraksjon
subtraction_1 = 120 - 50
subtraction_2 = 1200 - 500

# Multiplikasjon
multiplication_1 = 42.35 * 10
multiplication_2 = 348 * 1000

# Divisjon
division_1 = 225 / 100
division_2 = 7.75 / 1000

# Utskrift av resultatene med avrunding :.0f betyr feks ingen desimaler, :.1f betyr 1 desimal osv
print(f"70 kr + 50 kr = {addition_1:.0f} kr")
print(f"700 kr + 500 kr = {addition_2:.0f} kr")
print(f"120 kr - 50 kr = {subtraction_1:.0f} kr")
print(f"1200 kr - 500 kr = {subtraction_2:.0f} kr")
print(f"42.35 * 10 = {multiplication_1:.1f}")
print(f"348 * 1000 = {multiplication_2:.0f}")
print(f"225 / 100 = {division_1:.2f}")
print(f"7.75 / 1000 = {division_2:.5f}")

In [ ]:
# 1 Grunnleggende regning: 1.3 Overslagsregning
import math

def round_up(number, level):
    return math.ceil(number / level) * level

def round_down(number, level):
    return math.floor(number / level) * level

def round_nearest(number, level):
    return round(number / level) * level

def estimate(a, b, operation):
    if operation == '+':
        a_rounded = round_down(a, 10)
        b_rounded = round_up(b, 10)
        result = a_rounded + b_rounded
    elif operation == '-':
        a_rounded = round_up(a, 10)
        b_rounded = round_up(b, 10)
        result = a_rounded - b_rounded
    elif operation == '*':
        a_rounded = round_up(a, 10)
        b_rounded = round_down(b, 10)
        result = a_rounded * b_rounded
    elif operation == '/':
        a_rounded = round_down(a, 10)
        b_rounded = round_down(b, 1)
        result = a_rounded / b_rounded
    else:
        raise ValueError("Ugyldig operasjon")
    
    return a_rounded, b_rounded, result

def run_examples():
    examples = [
        (184.75, 257.20, '+'),
        (657.50, 379.45, '-'),
        (18.5, 26.3, '*'),
        (122, 3.12, '/'),
        (4.8, 14.18, '*'),
        (0.23, 18, '*'),
        (180, 47, '/')
    ]

    for a, b, op in examples:
        a_r, b_r, res = estimate(a, b, op)
        print(f"{a} {op} {b} ≈ {a_r} {op} {b_r} = {round(res)}")

def user_input():
    try:
        a = float(input("Skriv inn det første tallet: "))
        b = float(input("Skriv inn det andre tallet: "))
        op = input("Velg regneoperasjon (+, -, *, /): ")
        a_r, b_r, res = estimate(a, b, op)
        print(f"\nOverslag: {a} {op} {b} ≈ {a_r} {op} {b_r} = {round(res)}")
    except Exception as e:
        print(f"Feil: {e}")

def main():
    while True:
        print("\n--- OVERSLAGSREGNING ---")
        print("1. Kjør eksempler")
        print("2. Gjør egne beregninger")
        print("3. Avslutt")
        valg = input("Velg et alternativ (1-3): ")

        if valg == '1':
            run_examples()
        elif valg == '2':
            user_input()
        elif valg == '3':
            print("Avslutter programmet.")
            break
        else:
            print("Ugyldig valg. Prøv igjen.")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.5 Regne med brøk
import ipywidgets as widgets
from IPython.display import display, clear_output
from fractions import Fraction
import math
import re
import ast
import operator

# GUI-komponenter
num_terms = widgets.IntSlider(value=2, min=2, max=6, description="Antall ledd:")
confirm_button = widgets.Button(description="OK")
input_container = widgets.VBox()
calculate_button = widgets.Button(description="Beregn")
output = widgets.Output()

term_inputs = []
operator_inputs = []

# Evaluer komplekse uttrykk i brøker
def eval_expr(expr):
    expr = expr.replace('^', '**')
    expr = re.sub(r'√(\d+)', r'math.sqrt(\1)', expr)
    expr = expr.replace('sqrt', 'math.sqrt')

    ops = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg
    }

    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        elif isinstance(node, ast.BinOp):
            left = _eval(node.left)
            right = _eval(node.right)
            res = ops[type(node.op)](left, right)
            return Fraction(str(res)).limit_denominator(10**6)
        elif isinstance(node, ast.UnaryOp):
            operand = _eval(node.operand)
            return ops[type(node.op)](operand)
        elif isinstance(node, ast.Call):
            if isinstance(node.func, ast.Attribute) and node.func.attr == 'sqrt':
                val = math.sqrt(float(_eval(node.args[0])))
                return Fraction(str(val)).limit_denominator(10**6)
            else:
                raise TypeError("Ugyldig funksjon")
        elif isinstance(node, ast.Constant):
            return Fraction(str(node.value))
        elif isinstance(node, ast.Num):
            return Fraction(str(node.n))
        else:
            raise TypeError(f"Ugyldig uttrykk: {node}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

# Opprett felt for ledd
def create_inputs(_):
    global term_inputs, operator_inputs
    term_inputs = []
    operator_inputs = []
    children = []

    for idx in range(num_terms.value):
        term_type = widgets.Dropdown(options=["Heltall", "Desimaltall", "Brøk", "Kompleks brøk"], description=f"Ledd {idx+1}:")
        sign = widgets.Dropdown(options=['+', '-'], description="Fortegn:")

        int_input = widgets.IntText(description="Heltall")
        float_input = widgets.FloatText(description="Desimal")
        br_num = widgets.IntText(description="Teller")
        br_den = widgets.IntText(description="Nevner", value=1)
        k_num_expr = widgets.Text(description="Telleruttrykk")
        k_den_expr = widgets.Text(description="Nevneruttrykk")

        container = widgets.VBox()

        # Lager separat funksjon for å unngå referanseproblem
        def make_updater(container_ref, type_widget, sign_widget, i_widget, f_widget, bn_widget, bd_widget, kn_widget, kd_widget):
            def update_term_fields(change=None):
                if type_widget.value == "Heltall":
                    container_ref.children = [sign_widget, type_widget, i_widget]
                elif type_widget.value == "Desimaltall":
                    container_ref.children = [sign_widget, type_widget, f_widget]
                elif type_widget.value == "Brøk":
                    container_ref.children = [sign_widget, type_widget, bn_widget, bd_widget]
                elif type_widget.value == "Kompleks brøk":
                    container_ref.children = [sign_widget, type_widget, kn_widget, kd_widget]
            return update_term_fields

        updater = make_updater(container, term_type, sign, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr)
        term_type.observe(updater, names="value")
        updater()

        term_inputs.append((sign, term_type, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr))
        children.append(container)

        if idx < num_terms.value - 1:
            op = widgets.Dropdown(options=['+', '-', '*', '/'], description=f"Operator {idx+1}:")
            operator_inputs.append(op)
            children.append(op)

    children.append(calculate_button)
    children.append(output)
    input_container.children = children

# Konverter et ledd til Fraction
def to_fraction(sign, typ, i, f, bn, bd, kn_expr, kd_expr):
    if typ == "Heltall":
        val = Fraction(i)
    elif typ == "Desimaltall":
        val = Fraction(str(f)).limit_denominator()
    elif typ == "Brøk":
        if bd == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = Fraction(bn, bd)
    elif typ == "Kompleks brøk":
        teller = eval_expr(kn_expr)
        nevner = eval_expr(kd_expr)
        if nevner == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = teller / nevner
    else:
        raise ValueError(f"Ukjent type: {typ}")
    return val if sign == '+' else -val

# Kalkuler hele uttrykket
def calculate(_):
    with output:
        output.clear_output()
        try:
            sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[0]
            result = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
            expr_str = f"({result})"

            for idx, op_widget in enumerate(operator_inputs):
                op = op_widget.value
                sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[idx + 1]
                next_val = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
                expr_str += f" {op} ({next_val})"

                if op == '+':
                    result += next_val
                elif op == '-':
                    result -= next_val
                elif op == '*':
                    result *= next_val
                elif op == '/':
                    if next_val == 0:
                        raise ZeroDivisionError("Kan ikke dele på null")
                    result /= next_val

            desimal = round(float(result), 2)

            if abs(result.numerator) > result.denominator:
                heltall = result.numerator // result.denominator
                rest = abs(result.numerator) % result.denominator
                blandet = f"{heltall} {rest}/{result.denominator}" if rest else str(heltall)
            else:
                blandet = str(result)

            print(f"Uttrykk: {expr_str}")
            print(f"Forenklet brøk: {result}")
            print(f"Blandet tall: {blandet}")
            print(f"Desimaltall: {desimal}")

        except Exception as e:
            print(f"Feil: {e}")

# Koble knapper
confirm_button.on_click(create_inputs)
calculate_button.on_click(calculate)

# Startvisning
display(widgets.HBox([num_terms, confirm_button]))
display(input_container)

In [ ]:
# Omgjøringskalkulator mellom desimaltall, brøk og prosent
from decimal import Decimal, getcontext
from math import gcd

getcontext().prec = 10  # Sett presisjon for desimaler

def hovedmeny():
    print("Velkommen til omgjøringskalkulator mellom desimaltall, brøk og prosent ")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva vil du beregne?")
    print("1. Prosent til desimaltall og brøk")
    print("2. Brøk til desimaltall og prosent")
    print("3. Desimaltall til brøk og prosent")

def beregn_prosentandel(delen: float, hele: float) -> float:
    if hele == 0:
        raise ValueError("Hele kan ikke være null.")
    return (delen / hele) * 100

def decimal_to_fraction_and_percent(digits: str):
    try:
        n = Decimal(digits)
    except InvalidOperation:
        print("Ugyldig desimaltall.")
        return

    exponent = len(digits.split('.')[1]) if '.' in digits else 0
    numerator = int(n * 10**exponent)
    denominator = 10**exponent
    percent = float(n * 100)
    factor = gcd(numerator, denominator)
    num = numerator // factor
    den = denominator // factor

    print(f"Desimaltallet er {round(n, 3)}")
    print(f"Brøken er {num} / {den}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def percent_to_decimal_and_fraction(percent: float):
    decimal = percent / 100
    digits = str(decimal)
    decimal_to_fraction_and_percent(digits)

def fraction_to_decimal_and_percent(numerator: int, denominator: int):
    if denominator == 0:
        print("Nevneren kan ikke være null.")
        return
    decimal = Decimal(numerator) / Decimal(denominator)
    percent = float(decimal * 100)

    print(f"Desimaltallet er {round(decimal, 3)}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def main():
    while True:
        hovedmeny()
        choice = input("Velg et alternativ (1/2/3): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break
        elif choice == '1':
            percent = float(input("Skriv inn prosentverdien: "))
            percent_to_decimal_and_fraction(percent)
        elif choice == '2':
            numerator = int(input("Skriv inn telleren til brøken, altså det øverste tallet: "))
            denominator = int(input("Skriv inn nevneren til brøken, altså det nederste tallet: "))
            fraction_to_decimal_and_percent(numerator, denominator)
        elif choice == '3':
            digits = input("Skriv inn ett desimaltall, husk punktum, for å konvertere til brøk og prosent: ")
            decimal_to_fraction_and_percent(digits)
        else:
            print("Ugyldig valg. Vennligst prøv igjen.")
        
        restart = input("Vil du starte på nytt? (ja/nei): ").strip().lower()
        if restart == 'q':
            print("Programmet avsluttes.")
            break
        elif restart != 'ja':
            break

if __name__ == "__main__":
    main()

In [ ]:
# Forholdskalkulator med saft og vann som eksempel
from fractions import Fraction

# 🔧 Konverteringsfunksjon for mengde-input
def les_mengde(prompt):
    while True:
        tekst = input(f"{prompt} (f.eks. 1.5l, 15dl, 100cl, 250ml, 1/2l) eller q for å avslutte: ").strip().lower()
        if tekst == "q":
            print("\n📤 Programmet avsluttes. Takk for at du brukte kalkulatoren! 🧃")
            return None
        try:
            if tekst.endswith("l") and not tekst.endswith("ml"):
                mengde = Fraction(tekst[:-1]) * 10         # liter til dl
            elif tekst.endswith("dl"):
                mengde = Fraction(tekst[:-2])              # dl
            elif tekst.endswith("cl"):
                mengde = Fraction(tekst[:-2]) / 10         # cl til dl
            elif tekst.endswith("ml"):
                mengde = Fraction(tekst[:-2]) / 100        # ml til dl
            else:
                mengde = Fraction(tekst)                   # tolkes som dl
            return float(mengde)
        except:
            print("❌ Ugyldig mengde. Prøv igjen.")

# 📦 Les forhold saft:vann
def les_forhold():
    while True:
        forhold = input("Oppgi forhold mellom saft og vann (f.eks. 1:5 eller 1/6:5) eller q for å avslutte: ").strip()
        if forhold.lower() == "q":
            print("\n📤 Programmet avsluttes.")
            return None
        try:
            saft_del, vann_del = forhold.split(":")
            saft = Fraction(saft_del)
            vann = Fraction(vann_del)
            return saft, vann
        except:
            print("❌ Ugyldig format. Bruk f.eks. 1:5 eller 1/3:2")

# 🧾 Vis resultat i flere enheter
def vis_resultat(saft_dl, vann_dl):
    total_dl = saft_dl + vann_dl
    print(f"\n🧮 Resultat:")
    print(f"- Saft: {saft_dl:.2f} dl ({saft_dl/10:.2f} l, {saft_dl*10:.0f} cl, {saft_dl*100:.0f} ml)")
    print(f"- Vann: {vann_dl:.2f} dl ({vann_dl/10:.2f} l, {vann_dl*10:.0f} cl, {vann_dl*100:.0f} ml)")
    print(f"- Totalt: {total_dl:.2f} dl ({total_dl/10:.2f} l)")

# 🔢 Kalkulasjoner
def beregn_fra_totalmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    total = les_mengde("Hvor mye ferdig drikke ønsker du")
    if total is None:
        return
    total_deler = saft + vann
    saft_dl = (saft / total_deler) * total
    vann_dl = (vann / total_deler) * total
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_saftmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    saft_dl = les_mengde("Hvor mye saft har du")
    if saft_dl is None:
        return
    faktor = saft_dl / float(saft)
    vann_dl = float(vann) * faktor
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_vannmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    vann_dl = les_mengde("Hvor mye vann har du")
    if vann_dl is None:
        return
    faktor = vann_dl / float(vann)
    saft_dl = float(saft) * faktor
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_prosent():
    prosent_saft = input("Hvor mange prosent av drikken skal være saft? (f.eks. 20) eller q for å avslutte: ")
    if prosent_saft.lower() == "q":
        print("\n📤 Programmet avsluttes.")
        return
    try:
        prosent_saft = float(prosent_saft)
        total = les_mengde("Hvor mye ferdig drikke ønsker du")
        if total is None:
            return
        saft_dl = (prosent_saft / 100) * total
        vann_dl = total - saft_dl
        vis_resultat(saft_dl, vann_dl)
    except:
        print("❌ Ugyldig prosent. Prøv igjen.")

# 💰 Fordeling av beløp etter brøk
def fordel_beløp():
    print("\n💰 Fordeling av beløp mellom tre personer der to får oppgitt brøk og siste får resten.")
    try:
        total = float(input("Hvor mye penger skal fordeles totalt?: "))
        andel1 = Fraction(input("Hvor stor andel skal første person ha?: "))
        andel2 = Fraction(input("Hvor stor andel skal andre person ha?: "))
        if andel1 + andel2 > 1:
            print("❌ Summen av andelene er mer enn 1. Prøv igjen.")
            return
        andel3 = 1 - andel1 - andel2
        beløp1 = total * float(andel1)
        beløp2 = total * float(andel2)
        beløp3 = total * float(andel3)
        print(f"\n📊 Fordeling:")
        print(f"- Person 1 ({andel1}): {beløp1:.2f} kr")
        print(f"- Person 2 ({andel2}): {beløp2:.2f} kr")
        print(f"- Person 3 ({andel3}): {beløp3:.2f} kr")
    except:
        print("❌ Ugyldig input. Prøv igjen.")

# 🧃 Hovedmeny
def hovedprogram():
    while True:
        print("\n🧃 Forholdskalkulator for saft og vann 🧃")
        print("1. Beregn saft og vann fra forhold og total mengde")
        print("2. Beregn vann og total mengde fra forhold og saftmengde")
        print("3. Beregn saft og total mengde fra forhold og vannmengde")
        print("4. Beregn mengder fra ønsket prosent saft og total mengde")
        print("5. Fordel et pengebeløp etter brøker")
        print("6. Avslutt")
        valg = input("Velg et alternativ (1-6) eller q for å avslutte: ").strip().lower()

        if valg == "1":
            beregn_fra_totalmengde()
        elif valg == "2":
            beregn_fra_saftmengde()
        elif valg == "3":
            beregn_fra_vannmengde()
        elif valg == "4":
            beregn_fra_prosent()
        elif valg == "5":
            fordel_beløp()
        elif valg == "6" or valg == "q":
            print("\n📤 Programmet avsluttes. Takk for at du brukte kalkulatoren! 🧃")
            return
        else:
            print("❌ Ugyldig valg. Prøv igjen.")

        nytt = input("\n🔁 Vil du gjøre en ny beregning? (j/n): ").strip().lower()
        if nytt != "j":
            print("\n📤 Programmet avsluttes. Ha en fin dag!")
            return

# ▶️ Start programmet i Jupyter
hovedprogram()

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# 1.6 Prosent Finn % av ett tall. Formel: p % av ett tall = p/100 * tallet
while True:
    user_input = input("Skriv inn prosent (eller 'q' for å avslutte): ")
    if user_input.lower() == 'q':
        print("Avslutter programmet.")
        break
    try:
        prosent = float(user_input)
        hele_tallet = float(input("Skriv inn hele tallet: "))
        prosentdelen = prosent / 100 * hele_tallet
        print(f"{prosent}% av {hele_tallet} er {round(prosentdelen, 2)}")
    except ValueError:
        print("Ugyldig input. Vennligst skriv inn tallverdier.")

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# Hvor mange prosent delen av ett tall er av det hele tallet er, feks 10 er ...% av 30
from dataclasses import dataclass

@dataclass
class Prosentandel:
    delen: float
    hele: float

    def beregn(self) -> float:
        if self.hele == 0:
            raise ValueError("Hele kan ikke være null.")
        return (self.delen / self.hele) * 100

def main():
    print("Velkommen til programmet som regner ut hvor mange prosent ett tall er av ett annet tall")
    print("Trykk 'q' når som helst for å avslutte programmet.\n")

    while True:
        delen_input = input("Skriv inn delen av tallet (eller 'q' for å avslutte): ").strip().lower()
        if delen_input == 'q':
            print("Programmet avsluttes.")
            break

        hele_input = input("Skriv inn det hele tallet (eller 'q' for å avslutte): ").strip().lower()
        if hele_input == 'q':
            print("Programmet avsluttes.")
            break

        try:
            delen = float(delen_input)
            hele = float(hele_input)
            prosentandel = Prosentandel(delen=delen, hele=hele)
            prosent = prosentandel.beregn()
            print(f"{prosentandel.delen} er {prosent:.2f}% av {prosentandel.hele}.")
        except ValueError as e:
            print(e)
        except Exception as e:
            print(f"Ugyldig input: {e}")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# Regel 1. Del av tallet = (Hele tallet ∙ Prosenten) / 100
def beregn_hele_tallet(prosent, kroner):
    return kroner * 100 / prosent

def beregn_prosent(hele_tallet, kroner):
    return (kroner / hele_tallet) * 100

def beregn_del_av_tallet(hele_tallet, prosent):
    return hele_tallet * prosent / 100

def hovedprogram():
    print("Velkommen til kalkulatoren for: Del av tallet = (Hele tallet ∙ Prosenten) / 100")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva ønsker du å beregne?")
    
    print("1: Hele tallet")
    print("2: Prosent")
    print("3: Del av tallet")
    
    while True:
        valg = input("Skriv inn nummeret på det du ønsker å beregne (1, 2, eller 3): ")
        
        if valg.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break
        
        if valg == "1":
            try:
                prosent = float(input("Skriv inn prosentverdien: "))
                kroner = float(input("Skriv inn del av hele tallet: "))
                hele_tallet = beregn_hele_tallet(prosent, kroner)
                print(f"Hele tallet er: {hele_tallet:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "2":
            try:
                hele_tallet = float(input("Skriv inn hele tallet: "))
                kroner = float(input("Skriv inn del av tallet: "))
                prosent = beregn_prosent(hele_tallet, kroner)
                print(f"Prosentverdien er: {prosent:.2f} %\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "3":
            try:
                hele_tallet = float(input("Skriv inn hele tallet: "))
                prosent = float(input("Skriv inn prosentverdien: "))
                del_av_tallet = beregn_del_av_tallet(hele_tallet, prosent)
                print(f"Del av tallet er: {del_av_tallet:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        else:
            print("Ugyldig valg. Vennligst velg 1, 2, eller 3.")

# Kjør hovedprogrammet
hovedprogram()

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# Finn endringen i prosentpoeng mellom to tall
while True:
    user_input = input("Skriv inn startprosent (eller 'q' for å avslutte): ")
    if user_input.lower() == 'q':
        print("Avslutter programmet.")
        break
    try:
        startprosent = float(user_input)
        sluttprosent = float(input("Skriv inn sluttprosent: "))
        prosentpoeng = sluttprosent - startprosent
        prosent = prosentpoeng / startprosent * 100
        print(f"Endringen i prosentpoeng er: {round(prosentpoeng, 2)}")
        print(f"Endringen i prosent er: {round(prosent, 2)}%")
    except ValueError:
        print("Ugyldig input. Vennligst skriv inn tallverdier.")

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 2. Endringen i prosent = (Ny verdi – Opprinnelig verdi)/(Opprinnelig verdi) ∙ 100 %
def beregn_prosentendring(ny_verdi=None, opprinnelig_verdi=None, endring_i_prosent=None):
    try:
        if endring_i_prosent is not None and opprinnelig_verdi is not None:
            ny_verdi = opprinnelig_verdi * (1 + endring_i_prosent / 100)
            return ny_verdi
        elif ny_verdi is not None and opprinnelig_verdi is not None:
            endring_i_prosent = ((ny_verdi - opprinnelig_verdi) / opprinnelig_verdi) * 100
            return endring_i_prosent
        elif ny_verdi is not None and endring_i_prosent is not None:
            opprinnelig_verdi = ny_verdi / (1 + endring_i_prosent / 100)
            return opprinnelig_verdi
        else:
            return "Ugyldig input. Vennligst oppgi to av de tre verdiene."
    except ZeroDivisionError:
        return "Opprinnelig verdi kan ikke være null."
    except Exception as e:
        return f"En feil oppstod: {e}"

def hovedmeny():
    print("Velkommen til kalkulatoren for: Endringen i prosent = (Ny verdi – Opprinnelig verdi)/(Opprinnelig verdi) ∙ 100 %")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva vil du beregne?")
    
    print("1: Endringen i prosent")
    print("2: Ny verdi")
    print("3: Opprinnelig verdi")
    
    valg = input("Velg et alternativ (1, 2, 3): ")
    return valg
def hovedprogram():
    while True:
        valg = hovedmeny()
        
        if valg.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break
        
        if valg == "1":
            try:
                ny_verdi = float(input("Oppgi ny verdi: "))
                if ny_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                opprinnelig_verdi = float(input("Oppgi opprinnelig verdi: "))
                prosentendring = beregn_prosentendring(ny_verdi=ny_verdi, opprinnelig_verdi=opprinnelig_verdi)
                print(f"Endringen i prosent: {prosentendring:.2f} %\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "2":
            try:
                opprinnelig_verdi = float(input("Oppgi opprinnelig verdi: "))
                if opprinnelig_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                endring_i_prosent = float(input("Oppgi endringen i prosent: "))
                ny_verdi = beregn_prosentendring(opprinnelig_verdi=opprinnelig_verdi, endring_i_prosent=endring_i_prosent)
                print(f"Ny verdi: {ny_verdi:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "3":
            try:
                ny_verdi = float(input("Oppgi ny verdi: "))
                if ny_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                endring_i_prosent = float(input("Oppgi endringen i prosent: "))
                opprinnelig_verdi = beregn_prosentendring(ny_verdi=ny_verdi, endring_i_prosent=endring_i_prosent)
                print(f"Opprinnelig verdi: {opprinnelig_verdi:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        else:
            print("Ugyldig valg. Vennligst velg enten 1, 2 eller 3.")

if __name__ == "__main__":
    hovedprogram()

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 3.1 Vekstfaktor - Prosentvis økning. VF = 1 + Prosenten som desimaltall
def beregn_vekstfaktor(prosent):
    """
    Beregner vekstfaktoren basert på en prosentvis økning.
    
    Args:
    prosent (float): Prosentvis økning
    
    Returns:
    float: Vekstfaktoren
    """
    return 1 + prosent / 100

def main():
    print("Dette programmet regner ut vekstfaktoren ved en økning på en bestemt prosent.")
    try:
        prosent = float(input("Skriv inn denne prosenten: "))
        vekstfaktor = beregn_vekstfaktor(prosent)
        print(f"Vekstfaktoren ved en økning på {prosent:.2f} % er {vekstfaktor:.2f}.")
    except ValueError:
        print("Vennligst skriv inn et gyldig tall for prosenten.")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 3.2 Vekstfaktor - Prosentvis nedgang. VF = 1 - Prosenten som desimaltall
def beregn_nedgangsfaktor(prosent):
    """
    Beregner vekstfaktoren basert på en prosentvis nedgang.
    
    Args:
    prosent (float): Prosentvis nedgang
    
    Returns:
    float: Nedgangsfaktoren, avrundet til tre desimaler
    """
    nedgangsfaktor = 1 - prosent / 100
    return round(nedgangsfaktor, 3)

def main():
    print("Dette programmet regner ut vekstfaktoren ved en nedgang på en bestemt prosent.")
    try:
        prosent = float(input("Skriv inn denne prosenten: "))
        nedgangsfaktor = beregn_nedgangsfaktor(prosent)
        print(f"Vekstfaktoren ved en nedgang på {prosent:.2f} prosent er {nedgangsfaktor:.2f}.")
    except ValueError:
        print("Vennligst skriv inn et gyldig tall for prosenten.")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 3. Ny verdi = Opprinnelig verdi * Vekstfaktor
import sys  # for exit() hvis ønskelig

def beregn_verdi(opprinnelig_verdi=None, vekstfaktor=None, ny_verdi=None):
    if opprinnelig_verdi is not None and vekstfaktor is not None:
        return opprinnelig_verdi * vekstfaktor
    elif ny_verdi is not None and vekstfaktor is not None:
        return ny_verdi / vekstfaktor
    elif ny_verdi is not None and opprinnelig_verdi is not None:
        return ny_verdi / opprinnelig_verdi
    else:
        return None

def få_input(prompt):
    verdi_input = input(prompt)
    if verdi_input.strip().lower() == 'q':
        print("Programmet avsluttes.")
        return None
    try:
        return float(verdi_input)
    except ValueError:
        print("Ugyldig tall. Prøv igjen.")
        return få_input(prompt)

def hovedprogram():
    print("Velkommen til kalkulatoren for:  Ny verdi = Opprinnelig verdi * Vekstfaktor")
    print("Formel: Ny verdi = Opprinnelig verdi * Vekstfaktor")
    print("Skriv 'q' for å avslutte og trykk enter.\n")

    print("Hva vil du finne?")
    print("1: Ny verdi")
    print("2: Opprinnelig verdi")
    print("3: Vekstfaktor")

    valg = input("Skriv 1, 2 eller 3: ").strip()
    if valg.lower() == 'q':
        print("Programmet avsluttes.")
        return

    if valg == '1':
        opprinnelig_verdi = få_input("Oppgi opprinnelig verdi: ")
        if opprinnelig_verdi is None:
            return
        vekstfaktor = få_input("Oppgi vekstfaktor: ")
        if vekstfaktor is None:
            return
        ny_verdi = beregn_verdi(opprinnelig_verdi, vekstfaktor)
        print(f"\nNy verdi = {ny_verdi:.2f}")

    elif valg == '2':
        ny_verdi = få_input("Oppgi ny verdi: ")
        if ny_verdi is None:
            return
        vekstfaktor = få_input("Oppgi vekstfaktor: ")
        if vekstfaktor is None:
            return
        opprinnelig_verdi = beregn_verdi(None, vekstfaktor, ny_verdi)
        print(f"\nOpprinnelig verdi = {opprinnelig_verdi:.2f}")

    elif valg == '3':
        opprinnelig_verdi = få_input("Oppgi opprinnelig verdi: ")
        if opprinnelig_verdi is None:
            return
        ny_verdi = få_input("Oppgi ny verdi: ")
        if ny_verdi is None:
            return
        vekstfaktor = beregn_verdi(opprinnelig_verdi, None, ny_verdi)
        prosent_endring = (vekstfaktor - 1) * 100
        print(f"\nVekstfaktor = {vekstfaktor:.3f} ({prosent_endring:+.1f} % endring)")

    else:
        print("Ugyldig valg. Vennligst start programmet på nytt.")

if __name__ == "__main__":
    hovedprogram()

# $\color{green}{\text{Kapittel 2 - Personlig økonomi}}$

# $\color{blue}{\text{Kapittel 3 - Formler og geometri}}$

# $\color{green}{\text{Kapittel 4 - Statistikk}}$

# $\color{red}{\text{Kapittel 5 - Yrkesøkonomi}}$

In [ ]:
# 5 Yrkesøkonomi: 5.2 Merverdiavgift
# 5.2 Merverdiavgift beregning: Pris uten MVA = Pris med MVA/Vekstfaktoren 
def beregn_pris_uten_mva(pris_med_mva, vekstfaktor):
    return pris_med_mva / vekstfaktor

def beregn_pris_med_mva(pris_uten_mva, vekstfaktor):
    return pris_uten_mva * vekstfaktor

def beregn_vekstfaktor(pris_med_mva, pris_uten_mva):
    return pris_med_mva / pris_uten_mva

def velg_vekstfaktor():
    print("Velg MVA-sats:")
    print("1. 25% (For de fleste varer eller tjenester)")
    print("2. 15% (For mat og drikke)")
    print("3. 12% (For persontransport, kinobilletter og utleie av rom)")
    print("4. 0% (Helsetjenester, undervisningstjenester og kulturelle tjenester)")
    
    valg = input("Velg et alternativ (1/2/3/4): ").strip()
    if valg == '1':
        return 1.25
    elif valg == '2':
        return 1.15
    elif valg == '3':
        return 1.12
    elif valg == '4':
        return 1.00
    else:
        print("Ugyldig valg. Standard vekstfaktor 1.25 (25%) brukes.")
        return 1.25

def hovedprogram():
    while True:
        print("\nVelkommen til MVA kalkulator!")
        print("Trykk 'q' for å avslutte programmet.\n")
       
        print("Velg en beregning:")
        print("1. Pris uten MVA")
        print("2. Pris med MVA")
        print("3. Vekstfaktoren")

        choice = input("Velg et alternativ (1/2/3/q): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break

        try:
            if choice == '1':
                pris_med_mva = float(input("\nOppgi prisen med MVA: "))
                vekstfaktor = velg_vekstfaktor()
                pris_uten_mva = beregn_pris_uten_mva(pris_med_mva, vekstfaktor)
                resultat = f"\nPrisen uten MVA er {round(pris_uten_mva, 2)}"
            elif choice == '2':
                pris_uten_mva = float(input("\nOppgi prisen uten MVA: "))
                vekstfaktor = velg_vekstfaktor()
                pris_med_mva = beregn_pris_med_mva(pris_uten_mva, vekstfaktor)
                mva_belop = pris_med_mva - pris_uten_mva
                resultat = (
                    f"\nPrisen med MVA er {round(pris_med_mva, 2)}"
                    f"\nMerverdiavgiften utgjør {round(mva_belop, 2)}"
                )
            elif choice == '3':
                pris_med_mva = float(input("\nOppgi prisen med MVA: "))
                pris_uten_mva = float(input("Oppgi prisen uten MVA: "))
                vekstfaktor = beregn_vekstfaktor(pris_med_mva, pris_uten_mva)
                resultat = f"\nVekstfaktoren er {round(vekstfaktor, 2)}"
            else:
                resultat = "\nUgyldig valg. Vennligst oppgi 1, 2 eller 3."
        except ValueError:
            resultat = "\nUgyldig inndata. Vennligst oppgi et tall."

        print(resultat)

if __name__ == "__main__":
    hovedprogram()